In [ ]:
# installing segmentation models Pytorch
!pip install ../input/segmentation-models-wheels/timm-0.3.2-py3-none-any.whl
!pip install ../input/segmentation-models-wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl
!pip install ../input/segmentation-models-wheels/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install ../input/segmentation-models-wheels/segmentation_models_pytorch-0.1.3-py3-none-any.whl

In [ ]:
import numpy as np
import rasterio
import cv2
import os
from rasterio.windows import Window
import tifffile
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from skimage import io,transform
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import cv2, os, time
from albumentations.pytorch import ToTensor
import matplotlib.pyplot as plt
import torch, numba
import warnings
warnings.filterwarnings("ignore")
import gc
from torch.utils.tensorboard import SummaryWriter
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
test_dir = '/kaggle/input/hubmap-kidney-segmentation/test'
image_files = [file for file in os.listdir(test_dir) if file.split('.')[1] == 'tiff']
image_files

In [ ]:
# for img in image_files:
#     image_ = tifffile.imread(os.path.join(test_dir,img))
#     print(image_.shape)

In [ ]:
def make_grid(shape, window=512, min_overlap=32):
    x, y = shape   #(36800, 43780)
    nx = x // (window - min_overlap) + 1   #36800//(512-32) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx, ny, 4), dtype=np.int64) 
    for i in range(nx):
        for j in range(ny):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx * ny, 4)




def make_grids(shape,window=512,overlap=0):
    '''
    takes input image shape and return slice coordinates. Input can be unpadded image
    Last leftout frame is handled in a way that an overlaping window is taken
    eg: if window size = 5 and last x1,x2 = 7,12 and length of x = 15, Then we will take last window as x1,x2=10,15
    so last 2 windows will be (7,12) and (10,15). Last window will be always overlaping in these cases.
    '''
    y,x = shape[0],shape[1]   #h,w
    nx,ny = x // window, y // window
    y_,x_ = y - (y%window),x - (x%window)  #if no leftout portion (x%window) will be zero
    base_x = np.linspace(0,x_,nx+1,dtype=np.int64)
    base_y = np.linspace(0,y_,ny+1,dtype=np.int64)
    x1,x2 = base_x[0:-1], base_x[1:]    
    y1,y2 = base_y[0:-1], base_y[1:]
    if overlap != 0:
        sub_x = np.arange(0,nx*overlap,overlap)
        x1,x2 = x1 - sub_x,x2 - sub_x 
        sub_y = np.arange(0,ny*overlap,overlap)
        y1,y2 = y1 - sub_y, y2 - sub_y    
        
    # Always the last window at the end will be overlapping.
    # finally we may be left with some frame which is smaller that window size.
    # So we will append one window at last
    if y%window != 0: #if reminder !=0 append last window because leftout frame is there
        y1,y2 = np.append(y1,[y - window]),np.append(y2,[y])
    if x%window !=0 :
        x1,x2 = np.append(x1,[x - window]),np.append(x2,[x])
    
    
    slices = np.zeros(shape=(len(x1),len(y1),4),dtype=np.int64)
    for i in range(len(x1)):
        for j in range(len(y1)):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]
    slices = slices.reshape(len(x1) * len(y1), 4)
    return slices


def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def load_ckp(checkpoint_fpath, model):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath,map_location=torch.device('cpu'))
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # return model, optimizer, epoch value, min validation loss 
    return model

In [ ]:
import segmentation_models_pytorch as smp

checkpoint_path = '/kaggle/input/unetefficientnetb4/FPNbestmodel.pt_34'

model = smp.FPN(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=None,     # use `imagenet` pretreined weights for encoder initialization
    in_channels=3,                  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
learning_rate = 1e-3
model.to(device) 
model.eval()
model = load_ckp(checkpoint_path, model)

In [ ]:
from albumentations import *

def get_transforms(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    list_transforms = []
    list_transforms.extend(
        [
            Resize(256,256),
            Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
                            )
    list_trfms = Compose(list_transforms)
    return list_trfms

@numba.njit()
def rle_numba(pixels):
    size = len(pixels)
    points = []
    if pixels[0] == 1: points.append(0)
    flag = True
    for i in range(1, size):
        if pixels[i] != pixels[i-1]:
            if flag:
                points.append(i+1)
                flag = False
            else:
                points.append(i+1 - points[-1])
                flag = True
    if pixels[-1] == 1: points.append(size-points[-1]+1)    
    return points

def rle_numba_encode(image):
    pixels = image.flatten(order = 'F')
    points = rle_numba(pixels)
    return ' '.join(str(x) for x in points)

In [ ]:
WINDOW = 256*4 #size of crop from large image 1024
tile_size = 256
MIN_OVERLAP = 100

start_time = time.time()
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
subm = {}
trfm = get_transforms()
model.eval()

for i, filename in enumerate(image_files):
    dataset = rasterio.open(os.path.join(test_dir, filename), transform = identity)
    print(f'Processing image { i + 1 } of shape : {dataset.shape}')
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
    preds = np.zeros(dataset.shape, dtype=np.uint8)
    for (x1,x2,y1,y2) in tqdm(slices, desc=f'{filename}'):
        image = dataset.read([1,2,3],
                    window=Window.from_slices((x1,x2),(y1,y2)))
       
        image = np.moveaxis(image, 0, -1)
        image = trfm(image=image)
        image = image['image'].unsqueeze(0)

        
        with torch.no_grad():
            score = model(image)

            score2 = model(torch.flip(image, [0, 3]))
            score2 = torch.flip(score2, [3, 0])[0][0]

            score3 = model(torch.flip(image, [1, 2]))
            score3 = torch.flip(score3, [2, 1])[0][0]

            score_mean = (score + score2 + score3) / 3.0
            
            pred = score_mean.sigmoid().detach().cpu().numpy()[0][0]

            score_sigmoid = cv2.resize(pred, (WINDOW, WINDOW))
            preds[x1:x2,y1:y2] = (score_sigmoid > 0.5).astype(np.uint8)
            
    subm[i] = {'id':filename.split('.')[0], 'predicted': rle_numba_encode(preds)}
    
    del preds
    gc.collect();
    
    print('done')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)
submission.head()